In [2]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
from datetime import datetime
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
from pathlib import Path
from dotenv import load_dotenv
from random import randint
import pandas as pd 
import re, os, time, sys, csv, json

from toolkit import ordergenerator as og
from toolkit import general_tools as gt


In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
# self.chrome_options.add_argument('--headless')
chrome_options.add_argument('--incognito')
driver = webdriver.Chrome(options=chrome_options)

In [4]:
url = "https://www.booking.com/searchresults.fr.html?ss=Abri%C3%A8s%2C+Provence-Alpes-C%C3%B4te+d%27Azur%2C+France&label=en-row-booking-desktop-v0aqgtf3mjWbjFaAWHbxPAS652796017659%3Apl%3Ata%3Ap1%3Ap2%3Aac%3Aap%3Aneg%3Afi%3Atikwd-65526620%3Alp9070175%3Ali%3Adec%3Adm&aid=2311236&lang=fr&sb=1&src_elem=sb&src=index&dest_id=-1406559&dest_type=city&ac_position=0&ac_click_type=b&ac_langcode=en&ac_suggestion_list_length=3&search_selected=true&search_pageview_id=e3503f3e764b0288&ac_meta=GhBlMzUwM2YzZTc2NGIwMjg4IAAoATICZW46BkFicmllc0AASgBQAA%3D%3D&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&checkin=2024-01-15&checkout=2024-01-16&selected_currency=EUR"

In [5]:
driver.get(url)

In [7]:
page = driver.page_source

In [8]:
soupe = BeautifulSoup(page, 'lxml')
cards = soupe.find_all('div', {'data-testid':"property-card"})

In [25]:
len(cards)

25

In [44]:
card = cards[7]

In [45]:
card.find('span', {'data-testid':'price-and-discounted-price', 'class':'f6431b446c fbfd7c1165 e84eb96b1f'}).text

'€\xa0530'

In [46]:
card.find('div', {'data-testid':"availability-rate-information"}).find('span', {'data-testid':'price-and-discounted-price', 'class':'f6431b446c fbfd7c1165 e84eb96b1f'}).text[1:].replace(u'\xa0', u'')

'530'

In [48]:
taxe_text = card.find('div', {'data-testid':"availability-rate-information"}).find('div', {'data-testid':'taxes-and-charges'}).text[1:].replace(u'\xa0', u'')
taxe = 0 if '€' not in taxe_text else int(taxe_text)
taxe

0

In [49]:
card.find('div', {'data-testid':"availability-single"}).find('span', class_='e8f7c070a7').text.strip().replace('\n', '')

AttributeError: 'NoneType' object has no attribute 'text'

In [9]:
next_button = soupe.find('button', {'class':"a83ed08757 c21c56c305 f38b6daa18 d691166b09 ab98298258 deab83296e bb803d8689 a16ddf9c57"})
if next_button:
    next_page = driver.find_element(By.XPATH, "//button[@aria-label='Page suivante']")
    next_page.click()
    WebDriverWait(driver, 1)